<div style="text-align:center"><span style="color:blue; font-family:Times New Roman; font-size:3em;"> Data modeling  </span></div>

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import warnings
warnings.filterwarnings("ignore")
#from sklearn.externals import joblib
import joblib
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras import backend as K
#K.set_image_dim_ordering('th')
import h5py
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import copy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape,Lambda, Input, RepeatVector
from keras.utils import plot_model
from keras import initializers

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">I load the train data (X) and the test data (X_test) which are already preprocessed in the previous stage. 
 </span></div>

In [ ]:
# Prediction process logic: 

# X: state of all the items user seen upto order 20th for user 15
# Y: whether reordered or no in order 21st for user 15 (one column from X)

# X_test: state of all the items seens upto order 21st for user 15
# Y_pred: probability for each item have been seen upto 21st for user 15. 
#         ---> This probability will be the recommendations for order 22nd

In [2]:
X       =pd.read_csv('X_v4.csv')
X_test  =pd.read_csv('X_test_v4.csv')

In [3]:
X=X.take(np.random.permutation(len(X)))  #shuffle rows in the X. as such no longer on a user level
Y=X['reordered']
X.drop(['reordered','user_id', 'product_id','Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
X_test.drop(['user_id', 'product_id','Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
X.columns

Index(['total_orders_user', 'unique_items_user', 'total_items_user',
       'average_items_per_order', 'average_days_per_order', 'appear_rate',
       'reorder_num', 'add_to_cart_order', 'total_order_minus_last_order',
       'last_order_ratio', 'aisle_id', 'department_id', 'user_num_product',
       'product_total_orders', 'product_total_reorders', 'reorder_ratio',
       'last_1st_order', 'last_2st_order', 'last_3st_order', 'last_4st_order',
       'last_5st_order', 'last_6st_order', 'last_7st_order', 'last_8st_order',
       'last_9st_order', 'last_10st_order', 'last_11st_order',
       'last_12st_order', 'last_13st_order', 'last_14st_order',
       'last_15st_order', 'last_16st_order', 'last_17st_order',
       'last_18st_order', 'last_19st_order', 'last_20st_order',
       'last_five_orders_sum', 'last_10_orders_sum', 'last_15_orders_sum',
       'last_20_orders_sum', 'SAIN_LIM_Point'],
      dtype='object')

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The train data is shuffled and divided into the train set (80%) and the validation set (20%) by preserving the same percentage for each target class as in the complete set. In order to train neural network, the data is normalized to be in the range between 0 and 1. 
 </span></div>

In [18]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5) # n_split = 5 to have 80% 20% split. now if n_splits = 2, then it is 50% 50% split
i=1
for train_index, valid_index  in skf.split(X, Y): #X, Y here are train..so for user 15 upto order # 21
    joblib.dump(X.values[train_index],'X_train_modify_{}_v4.pkl'.format(i))
    joblib.dump(Y.values[train_index],'Y_train_modify_{}_v4.pkl'.format(i))
    joblib.dump(X.values[valid_index],'X_valid_modify_{}_v4.pkl'.format(i))
    joblib.dump(Y.values[valid_index],'Y_valid_modify_{}_v4.pkl'.format(i))
    i += 1

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
The neural network structure consists of one input layer, six hidden layers and one output layer. The weights are initialized using small Gaussian random numbers. The Rectifier activation function is used for each hidden layer. Two Dropout layers are applied to prevent overfitting. I also add Batch Normalization to each hidden layer of the network to perform the normalization for each training mini-batch. Batch Normalization allows us to use much higher learning rates and increase model performance. 
The output layer has a single neuron and a sigmoid activation is used for purposes of probabilistic classification. Finally the logarithmic loss function (binary_crossentropy) during training is used for binary classification. The model uses the efficient Adamax optimization algorithm for gradient descent and accuracy metrics is evaluated when the model is trained. 
I use the class_weights in the model.fit method to handle the imbalanced train data.
The class weights are calculated by inversely proportional to class frequencies in the input data.
I train the network for 500 epochs, with 1000 samples per mini-batch. The model weights that give the best result (lowest loss in the validation set) are saved for model evaluation and prediction of the test data. 

 </span></div>

In [6]:
#plot_model(NNmodel, to_file='test1.png',show_shapes=True,show_layer_names=True)

In [3]:
# model generated from 

def NNmodel(X_scaled_train, y_train,X_scaled_valid, y_valid,optimizer,batch_size,nb_epoch, c1, c2, c3, c4,c5,c6,drop1, \
            drop2, init,testnumber):
    model=Sequential()
    model.add(Dense(c1, input_dim=X_train.shape[1], init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop1))
    model.add(Dense(c2, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop2))
    model.add(Dense(c3, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(c4, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(c5, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(c6, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, init = init, activation='sigmoid'))
  
    from sklearn.utils import class_weight
    class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer,metrics=['accuracy']) 
    filepath="model_{}.best_v4.hdf5".format(testnumber)
    saveBestModel = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    model.fit(X_scaled_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, class_weight=class_weight,
              validation_data=(X_scaled_valid, y_valid),callbacks=[saveBestModel],verbose=1) 
    return

In [22]:
def NNmodel_prediction(X_scaled_test,optimizer,batch_size,c1, c2, c3, c4,c5,c6,drop1, drop2, init,testnumber):
    model=Sequential()
    model.add(Dense(c1, input_dim=X_train.shape[1], init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop1))
    model.add(Dense(c2, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(drop2))
    model.add(Dense(c3, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(c4, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(c5, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(c6, init = init, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, init = init, activation='sigmoid'))
  
    filepath="model_{}.best_v4.hdf5".format(testnumber)
    model.load_weights(filepath)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer,metrics=['accuracy']) 
    y_pred_test = model.predict_proba(X_scaled_test, batch_size=batch_size, verbose=1)
    return y_pred_test

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
Five-fold train/validation combinations are iterated and trained in the neural network model. The test data is fed into the model which generates 5 predictions via the iterations. The final prediction for the test data is obtained by averaging all 5 predictions in order to reduce overfitting.

 </span></div>

In [23]:
y_pred_all=[]
for i in tqdm(range(1,6)):
    ## load data 
    X_train=joblib.load('X_train_modify_{}_v4.pkl'.format(i))
    Y_train=joblib.load('Y_train_modify_{}_v4.pkl'.format(i))
    X_valid=joblib.load('X_valid_modify_{}_v4.pkl'.format(i))
    Y_valid=joblib.load('Y_valid_modify_{}_v4.pkl'.format(i))
    ## scale data
    scaler=MinMaxScaler()
    X_scaled_train=scaler.fit_transform(X_train)
    X_scaled_valid=scaler.transform(X_valid)
    X_scaled_test=scaler.transform(X_test.values)
    ## train data
    NNmodel(X_scaled_train, Y_train,X_scaled_valid, Y_valid,'adamax',1000,100, \
            500, 400, 400,200, 150,150,0.4, 0.4, 'normal',i)
    ## predict data
    y_pred_test=NNmodel_prediction(X_scaled_test,'adamax', 1000,500, 400, 400,200, 150,150, 0.4, 0.4, 'normal',i)
    y_pred_all.append(y_pred_test)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Train on 919289 samples, validate on 229823 samples
Epoch 1/100
919289/919289 [==============================] - 84s 92us/step - loss: 0.2373 - accuracy: 0.9205 - val_loss: 0.2161 - val_accuracy: 0.9242

Epoch 00001: val_loss improved from inf to 0.21610, saving model to model_1.best_v4.hdf5
Epoch 2/100
919289/919289 [==============================] - 104s 113us/step - loss: 0.2097 - accuracy: 0.9274 - val_loss: 0.2086 - val_accuracy: 0.9276

Epoch 00002: val_loss improved from 0.21610 to 0.20863, saving model to model_1.best_v4.hdf5
Epoch 3/100
919289/919289 [==============================] - 82s 89us/step - loss: 0.2088 - accuracy: 0.9276 - val_loss: 0.2074 - val_accuracy: 0.9279

Epoch 00003: val_loss improved from 0.20863 to 0.20742, saving model to model_1.best_v4.hdf5
Epoch 4/100
919289/919289 [==============================] - 84s 91us/step - loss: 0.2082 - accuracy: 0.9276 - val_loss: 0.2078 - val_accuracy: 0.9276

Epoch 00004: val_loss did not improve from 0.20742
Epoch 5/100



Epoch 00038: val_loss did not improve from 0.20635
Epoch 39/100
919289/919289 [==============================] - 73s 80us/step - loss: 0.2049 - accuracy: 0.9283 - val_loss: 0.2073 - val_accuracy: 0.9281

Epoch 00039: val_loss did not improve from 0.20635
Epoch 40/100
919289/919289 [==============================] - 72s 78us/step - loss: 0.2047 - accuracy: 0.9284 - val_loss: 0.2075 - val_accuracy: 0.9282

Epoch 00040: val_loss did not improve from 0.20635
Epoch 41/100
919289/919289 [==============================] - 76s 82us/step - loss: 0.2048 - accuracy: 0.9283 - val_loss: 0.2069 - val_accuracy: 0.9281

Epoch 00041: val_loss did not improve from 0.20635
Epoch 42/100
919289/919289 [==============================] - 72s 78us/step - loss: 0.2046 - accuracy: 0.9284 - val_loss: 0.2067 - val_accuracy: 0.9282

Epoch 00042: val_loss did not improve from 0.20635
Epoch 43/100
919289/919289 [==============================] - 74s 81us/step - loss: 0.2048 - accuracy: 0.9284 - val_loss: 0.2067 - v

919289/919289 [==============================] - 74s 81us/step - loss: 0.2028 - accuracy: 0.9289 - val_loss: 0.2073 - val_accuracy: 0.9281

Epoch 00079: val_loss did not improve from 0.20635
Epoch 80/100
919289/919289 [==============================] - 69s 75us/step - loss: 0.2029 - accuracy: 0.9289 - val_loss: 0.2072 - val_accuracy: 0.9284

Epoch 00080: val_loss did not improve from 0.20635
Epoch 81/100
919289/919289 [==============================] - 74s 80us/step - loss: 0.2027 - accuracy: 0.9290 - val_loss: 0.2071 - val_accuracy: 0.9283

Epoch 00081: val_loss did not improve from 0.20635
Epoch 82/100
919289/919289 [==============================] - 74s 80us/step - loss: 0.2026 - accuracy: 0.9288 - val_loss: 0.2076 - val_accuracy: 0.9282

Epoch 00082: val_loss did not improve from 0.20635
Epoch 83/100
919289/919289 [==============================] - 71s 78us/step - loss: 0.2025 - accuracy: 0.9288 - val_loss: 0.2072 - val_accuracy: 0.9285

Epoch 00083: val_loss did not improve from 0

 20%|███████████████▌                                                              | 1/5 [2:03:56<8:15:45, 7436.33s/it]

...

 80%|██████████████████████████████████████████████████████████████▍               | 4/5 [8:04:50<2:01:30, 7290.23s/it]

Train on 919290 samples, validate on 229822 samples
Epoch 1/100
919290/919290 [==============================] - 76s 83us/step - loss: 0.2369 - accuracy: 0.9208 - val_loss: 0.2097 - val_accuracy: 0.9284

Epoch 00001: val_loss improved from inf to 0.20966, saving model to model_5.best_v4.hdf5
Epoch 2/100
919290/919290 [==============================] - 69s 75us/step - loss: 0.2097 - accuracy: 0.9272 - val_loss: 0.2138 - val_accuracy: 0.9240

Epoch 00002: val_loss did not improve from 0.20966
Epoch 3/100
919290/919290 [==============================] - 75s 81us/step - loss: 0.2087 - accuracy: 0.9274 - val_loss: 0.2095 - val_accuracy: 0.9267

Epoch 00003: val_loss improved from 0.20966 to 0.20946, saving model to model_5.best_v4.hdf5
Epoch 4/100
919290/919290 [==============================] - 69s 76us/step - loss: 0.2082 - accuracy: 0.9275 - val_loss: 0.2122 - val_accuracy: 0.9250

Epoch 00004: val_loss did not improve from 0.20946
Epoch 5/100
919290/919290 [=============================


Epoch 00039: val_loss did not improve from 0.20630
Epoch 40/100
919290/919290 [==============================] - 69s 75us/step - loss: 0.2048 - accuracy: 0.9281 - val_loss: 0.2066 - val_accuracy: 0.9288

Epoch 00040: val_loss did not improve from 0.20630
Epoch 41/100
919290/919290 [==============================] - 72s 79us/step - loss: 0.2048 - accuracy: 0.9283 - val_loss: 0.2067 - val_accuracy: 0.9285

Epoch 00041: val_loss did not improve from 0.20630
Epoch 42/100
919290/919290 [==============================] - 73s 80us/step - loss: 0.2047 - accuracy: 0.9283 - val_loss: 0.2064 - val_accuracy: 0.9287

Epoch 00042: val_loss did not improve from 0.20630
Epoch 43/100
919290/919290 [==============================] - 70s 76us/step - loss: 0.2046 - accuracy: 0.9283 - val_loss: 0.2069 - val_accuracy: 0.9287

Epoch 00043: val_loss did not improve from 0.20630
Epoch 44/100
919290/919290 [==============================] - 76s 83us/step - loss: 0.2047 - accuracy: 0.9283 - val_loss: 0.2066 - v

919290/919290 [==============================] - 73s 80us/step - loss: 0.2029 - accuracy: 0.9287 - val_loss: 0.2070 - val_accuracy: 0.9285

Epoch 00080: val_loss did not improve from 0.20630
Epoch 81/100
919290/919290 [==============================] - 75s 82us/step - loss: 0.2027 - accuracy: 0.9288 - val_loss: 0.2074 - val_accuracy: 0.9288

Epoch 00081: val_loss did not improve from 0.20630
Epoch 82/100
919290/919290 [==============================] - 70s 76us/step - loss: 0.2026 - accuracy: 0.9287 - val_loss: 0.2073 - val_accuracy: 0.9288

Epoch 00082: val_loss did not improve from 0.20630
Epoch 83/100
919290/919290 [==============================] - 72s 79us/step - loss: 0.2028 - accuracy: 0.9287 - val_loss: 0.2074 - val_accuracy: 0.9284

Epoch 00083: val_loss did not improve from 0.20630
Epoch 84/100
919290/919290 [==============================] - 70s 76us/step - loss: 0.2025 - accuracy: 0.9288 - val_loss: 0.2074 - val_accuracy: 0.9286

Epoch 00084: val_loss did not improve from 0

100%|███████████████████████████████████████████████████████████████████████████████| 5/5 [10:06:00<00:00, 7272.13s/it]


In [24]:
# y_pred_all here is state of 22nd order for user 15 and probability of purchaing 

y_pred_all=np.array(y_pred_all)
y_pred_average=np.average(y_pred_all, axis=0)

In [25]:
len(y_pred_average)

1178576

In [26]:
np.save('y_pred_average_v4.npy', y_pred_average)

In [28]:
y_pred_average

array([[0.18260631],
       [0.18450725],
       [0.05097334],
       ...,
       [0.00205428],
       [0.00280767],
       [0.00295321]], dtype=float32)

In [4]:
y_pred_average = np.load("y_pred_average_v4.npy")

In [5]:
# Append prob back to original x_test
X_test=pd.read_csv('X_test_v4.csv')
print(X_test.shape)
X_test["prediction_prob"] = y_pred_average
print(X_test.shape)

(1178576, 45)
(1178576, 46)


In [31]:
# two level of validation 
# 1st : is for binary classification prediction compare between train from train_set (X) and validation from train_set(X) 
# 2nd : is for prediction compare with Dream model for the best 10 selected 


In [6]:
X_test["prob_rank"] = X_test.groupby("user_id")["prediction_prob"].rank(ascending=False)
rank_df =  X_test[["user_id","product_id","prediction_prob","prob_rank"]]
rank_df = rank_df[rank_df.prob_rank <= 15]
rank_df.sort_values(["user_id","prob_rank"])
df_pred = copy.deepcopy(rank_df)

In [7]:
df_pred[df_pred.user_id == 11]

,user_id,product_id,prediction_prob,prob_rank
0,11,4238.0,0.182606,7.0
1,11,4310.0,0.184507,6.0
6,11,7118.0,0.121767,9.0
15,11,14531.0,0.179067,8.0
18,11,20035.0,0.059272,15.0
19,11,20213.0,0.424928,1.0
20,11,20460.0,0.268277,4.0
26,11,33482.0,0.069576,13.0
28,11,33947.0,0.356440,2.0
29,11,33982.0,0.206245,5.0


### Compare with last order in prior_train (actual data)

In [8]:
prior_train = pd.read_csv('prior_train_james_v3.csv') # prior_train_final v3 is okay
prior_train = prior_train[prior_train.eval_set != "test"]
user_maxorder_map = prior_train.groupby("user_id").agg({"order_number":"max"}).reset_index()
user_maxorder_map.rename(columns = {"order_number":"total_orders_user"}, inplace = True)
entire_df = pd.merge(prior_train,user_maxorder_map, on ="user_id", how = "left").sort_values(["user_id","order_number"])
df_act = entire_df[entire_df.order_number == entire_df.total_orders_user ][["user_id","product_id","order_number","total_orders_user"]].reset_index()
df_act #actual last order (i.e 22nd order for user 15)

,index,user_id,product_id,order_number,total_orders_user
0,2787881,11,21036.0,7,7
1,2787882,11,42094.0,7,7
2,2787883,11,5038.0,7,7
3,2787884,11,20680.0,7,7
4,2787885,11,54031.0,7,7
...,...,...,...,...,...
193798,3199351,206202,33302.0,22,22
193799,3199352,206202,32906.0,22,22
193800,3213988,206206,13044.0,67,67
193801,3213989,206206,24870.0,67,67


In [9]:
df_pred[df_pred.user_id == 11].sort_values("prob_rank")

,user_id,product_id,prediction_prob,prob_rank
19,11,20213.0,0.424928,1.0
28,11,33947.0,0.356440,2.0
45,11,56329.0,0.298721,3.0
20,11,20460.0,0.268277,4.0
29,11,33982.0,0.206245,5.0
1,11,4310.0,0.184507,6.0
0,11,4238.0,0.182606,7.0
15,11,14531.0,0.179067,8.0
6,11,7118.0,0.121767,9.0
41,11,53994.0,0.114840,10.0


In [10]:
# Evaluation matrix 
def cal_confidence_score(user_id, df_act,df_pred):
    
    actual_basket_food_list = df_act[df_act.user_id == user_id].product_id.tolist()
    pred_top_10_food_list    = df_pred[df_pred.user_id == user_id].product_id.tolist()
    
    if len(actual_basket_food_list) < 15:
        # evaluate whether top 10 items predicted are in actual basket 
        confi_score = len([i for i in pred_top_10_food_list if i in actual_basket_food_list])/len(actual_basket_food_list)
    else:
        confi_score = len([i for i in pred_top_10_food_list if i in actual_basket_food_list])/15

    return(confi_score)

In [11]:
cal_confidence_score(11, df_act,df_pred)
actual_basket_food_list = df_act[df_act.user_id == 11].product_id.tolist()
pred_top_10_food_list   = df_pred[df_pred.user_id == 11].product_id.tolist()
print(actual_basket_food_list)
print(pred_top_10_food_list)

[21036.0, 42094.0, 5038.0, 20680.0, 54031.0, 40984.0, 34211.0, 53470.0, 10488.0, 56329.0, 33982.0]
[4238.0, 4310.0, 7118.0, 14531.0, 20035.0, 20213.0, 20460.0, 33482.0, 33947.0, 33982.0, 34211.0, 43294.0, 53994.0, 55118.0, 56329.0]


In [12]:
cal_confidence_score(11,df_act,df_pred)

0.2727272727272727

In [13]:
user_confi_score = df_act.drop_duplicates(subset= ["user_id"], keep = "first")["user_id"].reset_index()
user_confi_score["confi_score"] = user_confi_score.user_id.apply(lambda x: cal_confidence_score(x, df_act,df_pred))

In [14]:
bins   = [0,0.2,0.4,0.6,0.8,1.0]
labels = ["0-0.2","0.2-0.4","0.4-0.6","0.6-0.8","0.8-1.0"]
user_confi_score['confi_bucket'] = pd.cut(user_confi_score["confi_score"], bins=bins, labels=labels, include_lowest=True)

In [15]:
confi_score_result = pd.DataFrame(user_confi_score.confi_bucket.value_counts()).reset_index()
confi_score_result.rename(columns = {"index":"confi_bucket","confi_bucket":"frequency"},inplace = True)
confi_score_result.sort_values("frequency",ascending = False) #average size of the basket # how many 

,confi_bucket,frequency
0,0.2-0.4,5039
1,0.4-0.6,4941
2,0-0.2,4082
3,0.8-1.0,3246
4,0.6-0.8,3079


In [18]:
confi_score_result["percent"] = round(confi_score_result.frequency/sum(confi_score_result.frequency),2)

In [19]:
confi_score_result

,confi_bucket,frequency,percent
0,0.2-0.4,5039,0.25
1,0.4-0.6,4941,0.24
2,0-0.2,4082,0.20
3,0.8-1.0,3246,0.16
4,0.6-0.8,3079,0.15


In [15]:
# v3 result without nutrition

confi_score_result = pd.DataFrame(user_confi_score.confi_bucket.value_counts()).reset_index()
confi_score_result.rename(columns = {"index":"confi_bucket","confi_bucket":"frequency"},inplace = True)
confi_score_result.sort_values("frequency",ascending = False) #average size of the basket # how many 

,confi_bucket,frequency
0,0-0.2,6495
1,0.2-0.4,5654
2,0.4-0.6,3956
3,0.8-1.0,2289
4,0.6-0.8,1994


In [52]:
user_confi_score

,index,user_id,confi_score,confi_bucket
0,0,11,0.272727,0.2-0.4
1,11,12,0.000000,0-0.2
2,20,14,0.416667,0.4-0.6
3,32,23,0.500000,0.4-0.6
4,42,34,1.000000,0.8-1.0
...,...,...,...,...
20383,193768,206168,0.800000,0.6-0.8
20384,193773,206177,0.250000,0.2-0.4
20385,193777,206182,0.307692,0.2-0.4
20386,193790,206202,0.700000,0.6-0.8


In [17]:
sampled_user = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Clustering/20kusers_new.csv",usecols = ["user_id","kmean_cust_type"])

In [19]:
df_merg = pd.merge(user_confi_score,sampled_user, on="user_id", how = "left")

In [21]:
user20k_confscore_label = df_merg[["user_id","confi_score","confi_bucket","kmean_cust_type"]]

In [22]:
user20k_confscore_label.to_csv("user20k_confscore_label.csv")

# Kaggle results 

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
In the Kaggle competition of Instacart Market Basket Analysis, the evaluation of the prediction results is based on the F1 score. The F1 score is a weighted average of precision and recall. The F1 score needs be maximized to ensure that both precision and recall are high. In order to maximize the F1 score, an appropriate probability threshold is set to predict the output labels of the test data. In the end, the prediction of the reordered products in the latest purchase of the test users is summited to Kaggle. The F1 score shown in the Leaderboard of Kaggle is 0.373. 
 </span></div>

In [ ]:
thresh=0.24
y_pred_binary=np.where(y_pred_average[:, 0]>thresh, 1,0)
X_test_new=X_test.copy()
X_test_new['pred']=y_pred_binary
X_test_new['prediction_pro']=y_pred_test[:, 0]
X_test_new['product_id']=X_test_new['product_id'].astype(int)
prediction=X_test_new[X_test_new.pred==1].groupby('user_id')['product_id'].\
apply(lambda x: ' '.join([str(e) for e in set(x)]))

In [ ]:
test = pd.read_csv("sample_submission.csv")
prior_train=pd.read_csv('prior_train_final.csv')
submission=pd.merge(test, prior_train,on='order_id', how='left')[['order_id', 'user_id']]
submission['products']=submission.user_id.map(prediction)
submission['products'].fillna('None', inplace=True)
submission[['order_id','products']].to_csv('submission.csv', index=False)

<div style="text-align:center"><span style="color:blue; font-family:Times New Roman; font-size:3em;"> Conclusion  </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
In this project, I have developed a neural network model to predict which products users will buy again in their next purchase. The NN model combined with feature engineering technique can well capture both sequential behavior and general taste of users, leading to 0.373 of the F1 score in the Kaggle competition of Instacart Market Basket Analysis. If a model ensembling technique (a neural network model + XGBoost ) is used, it can push the F1 score in the leaderboard to 0.383 and achieve the ranking of top 9% in this competition. 
 </span></div>

In [ ]:
X       =pd.read_csv('X.csv')
X_test  =pd.read_csv('X_test.csv')

In [ ]:
X=X.take(np.random.permutation(len(X))) #shuffle rows in the X. as such no longer on a user level
Y=X['reordered']
X.drop(['reordered','user_id', 'product_id','Unnamed: 0'], axis=1, inplace=True)
X_test.drop(['user_id', 'product_id','Unnamed: 0'], axis=1, inplace=True)
X.columns

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf_new = StratifiedKFold(n_splits=5)
i=1
for train_index_new, valid_index_new  in skf_new.split(X, Y): #X, Y here are train..so for user 15 upto order # 21
    joblib.dump(X.values[train_index_new],'X_train_new_modify_{}_ahhahaha.pkl'.format(i))
    joblib.dump(Y.values[train_index_new],'Y_train_new_modify_{}_ahhahaha.pkl'.format(i))
    joblib.dump(X.values[valid_index_new],'X_valid_new_modify_{}_ahhahaha.pkl'.format(i))
    joblib.dump(Y.values[valid_index_new],'Y_valid_new_modify_{}_ahhahaha.pkl'.format(i))
    i += 1

In [ ]:
X.shape

In [ ]:
print(train_index_new.shape)
train_index_new

In [ ]:
print(valid_index_new.shape)
valid_index_new

In [ ]:
X

In [ ]:
X.values